In [7]:
import os
import re
import sys
import warnings
import numpy as np
import pandas as pd

sys.path.append('../modules')
import estimate_theil_index as estimate_theil
import estimate_temporal_theil_scaling as estimate_tts

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [8]:
input_path_raw = "../input_files/raw_data"
input_path_processed = "../input_files/processed_data"
input_path_data_dictionary = "../input_files/data_dictionary"
log_path = "../logs"
output_path = "../output_files"
input_generation_date = "2023-06-08"

## Load time series

In [9]:
df_stock_indexes = pd.read_csv("{}/df_stock_index_{}.csv".format(input_path_processed, re.sub("-", "", input_generation_date)), low_memory = False)
df_stock_indexes = df_stock_indexes[((df_stock_indexes["symbol"] == "IMOEX.ME") & (df_stock_indexes["date"] >= "2021-01-01"))]

## Estimate Diffusive Algorithm - Stock indexes

In [10]:
df_dtts_stock_indexes = estimate_theil.estimate_diffusive_algorithm_global(
    df_data = df_stock_indexes,
    minimal_steps = 0,
    log_path = log_path,
    log_filename = "log_dtts_example",
    verbose = 0,
    tqdm_bar = True
)
df_dtts_stock_indexes

100%|████████████████████| 595/595 [00:01<00:00, 435.13it/s]


,level_0,symbol,time,sub_time,diffusive_log_return,diffusive_absolute_log_return,diffusive_log_volatility
0,0,IMOEX.ME,1926,1926,0.018523,0.018523,2.118718
1,1,IMOEX.ME,1927,1926,0.021098,0.021098,3.876704
2,2,IMOEX.ME,1928,1926,0.024629,0.024629,5.500466
3,3,IMOEX.ME,1929,1926,0.049181,0.049181,8.212376
4,4,IMOEX.ME,1930,1926,0.057155,0.057155,8.936621
...,...,...,...,...,...,...,...
177305,1,IMOEX.ME,2519,2518,-0.014158,0.014158,1.793809
177306,2,IMOEX.ME,2520,2518,-0.009158,0.019158,3.185973
177307,0,IMOEX.ME,2519,2519,-0.004796,0.004796,1.426586
177308,1,IMOEX.ME,2520,2519,0.000204,0.009796,2.818750


## Theil index estimation

In [11]:
df_theil_stock_indexes = estimate_theil.estimate_shannon_index_global(
    df_dtts_data = df_dtts_stock_indexes,
    minimal_steps = 0,
    log_path = log_path,
    log_filename = "log_theil_example",
    verbose = 0,
    tqdm_bar = True
)

df_theil_stock_indexes

100%|█████████████████████| 595/595 [00:14<00:00, 41.48it/s]


,symbol,time,time_series,mean_value,theil_index,shannon_index
0,IMOEX.ME,1926,log-return,0.018523,0.000000,0.000000
1,IMOEX.ME,1926,absolute log-return,0.018523,0.000000,0.000000
2,IMOEX.ME,1926,log-return volatility,2.118718,0.000000,0.000000
3,IMOEX.ME,1927,log-return,0.011837,0.349178,0.343970
4,IMOEX.ME,1927,absolute log-return,0.011837,0.349178,0.343970
...,...,...,...,...,...,...
1780,IMOEX.ME,2519,absolute log-return,3.536602,0.269383,6.117496
1781,IMOEX.ME,2519,log-return volatility,523.838796,0.210457,6.176422
1782,IMOEX.ME,2520,log-return,-0.075515,3.090114,3.298447
1783,IMOEX.ME,2520,absolute log-return,3.535658,0.270030,6.118531


## Estimate Temporal Theil Scaling exponent

In [12]:
df_stock_tts = estimate_tts.estimate_tts_parameters(
    df_fts = df_theil_stock_indexes,
    minimal_steps = 60,
    p_norm = 2,
    log_path = log_path,
    log_filename = "tts_evolution_example",
    verbose = 0,
    tqdm_bar = True
)
df_stock_tts

100%|█████████████████████| 595/595 [00:07<00:00, 84.48it/s]


,symbol,max_step,time_series,p_norm,coefficient_tts,error_coefficient_tts,exponent_tts,error_exponent_tts,average_error_tts,rsquared_tts
0,IMOEX.ME,1926,log-return,2,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
1,IMOEX.ME,1926,absolute log-return,2,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
2,IMOEX.ME,1926,log-return volatility,2,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
3,IMOEX.ME,1927,absolute log-return,2,-2.181144e-12,inf,1.000000e+00,inf,0.500001,-1.000000
4,IMOEX.ME,1927,log-return,2,-2.181144e-12,inf,1.000000e+00,inf,0.500001,-1.000000
...,...,...,...,...,...,...,...,...,...,...
1780,IMOEX.ME,2519,log-return,2,6.199660e-01,0.149088,2.161229e-08,0.276535,0.550898,-0.000887
1781,IMOEX.ME,2519,log-return volatility,2,1.009112e+00,0.001813,1.850909e-01,0.001655,0.130425,0.968477
1782,IMOEX.ME,2520,absolute log-return,2,9.922525e-01,0.002854,1.500192e-01,0.002314,0.171457,0.906662
1783,IMOEX.ME,2520,log-return,2,6.199031e-01,0.149099,1.040534e-06,0.276740,0.550439,-0.000887
